In [11]:
from importlib import reload

import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
sess = tf.InteractiveSession()

Extracting MNIST_data/train-images-idx3-ubyte.gz


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [14]:
import ConvNetMnist as CNM
reload(CNM)

net = CNM.ConvNet(sess)
net.train(mnist, n_iter=200)

step 0, training accuracy 0.12


step 100, training accuracy 0.82


test accuracy 0.8992


In [21]:
N = 10
# get N images of '2' from the dataset correctly predicted by the dataset
i = 0
res = []
for (idx, num) in enumerate(mnist.test.labels):
    if num[2] == 1:  # labels are 1-hot encoded
        # check if correctly predicted
        x = mnist.test.images[idx][None, :]
        y = mnist.test.labels[idx][None, :]
        correct = net.test_accuracy(x, y)
        if correct != 1:
            break
        res.append(idx)
        i += 1
        if i >= N:
            break

rows = np.array(res)
twos_x = mnist.test.images[rows, :]
twos_y = mnist.test.labels[rows, :]

In [141]:
# Restrict the cross entropy calculation to only 2 and 6 labels.
# Then increasing cross entropy is equivalent to moving towards label 6.

# y_conv_2_6 = tf.transpose(tf.gather(tf.transpose(y_conv), indices=[2,6]))
# y_2_6 = tf.transpose(tf.gather(tf.transpose(y_), indices=[2,6]))
# 
# cross_entropy_2_6 = tf.reduce_mean(
#     tf.nn.softmax_cross_entropy_with_logits(logits=y_conv_2_6, labels=y_2_6)
# )
# 
# x_grad = tf.gradients(cross_entropy_2_6, x)[0]
# y_pred = tf.argmax(y_conv,1)
# 
# fool_x = twos_x
# for i in range(3):
#     x_grad_vals = x_grad.eval(feed_dict = {
#        x: fool_x,
#       y_: twos_y,
#      keep_prob: 1.0
#     })
# 
#     eps = 1
#     fool_x = fool_x + eps * np.sign(x_grad_vals)
#     new_pred = y_pred.eval(feed_dict = {
#         x: fool_x,
#         keep_prob: 1.0
#     })
#     print(new_pred)

/home/michbad/anaconda3/envs/tensorflow/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[6 6 5 1 4 6 1 6 5 6]
[6 6 6 6 6 6 6 6 6 6]
[6 6 6 6 6 6 6 6 6 6]


In [139]:
x.value

AttributeError: 'Tensor' object has no attribute 'value'